In [14]:
import copy

class CliffWalkingEnv:
    """ 悬崖漫步环境"""
    def __init__(self, ncol=12, nrow=4):
        self.ncol = ncol  # 定义网格世界的列
        self.nrow = nrow  # 定义网格世界的行
        # 转移矩阵P[state][action] = [(p, next_state, reward, done)]包含下一个状态和奖励
        self.P = self.createP()

    def createP(self):
        # 初始化
        P = [[[] for j in range(4)] for i in range(self.nrow * self.ncol)]
        ## 或者 P = np.zeros((self.nrow * self.ncol, 2 * 2))
        # 4种动作, change[0]:上,change[1]:下, change[2]:左, change[3]:右。坐标系原点(0,0)
        # 定义在左上角
        change = [[0, -1], [0, 1], [-1, 0], [1, 0]]
        for i in range(self.nrow):
            for j in range(self.ncol):
                for a in range(4):
                    #   位置在悬崖或者目标状态,因为无法继续交互,任何动作奖励都为0
                    ##  最后一行是悬崖的，处在悬崖是不可能的，所以可以任意初始化 
                    if i == self.nrow - 1 and j > 0:
                        P[i * self.ncol + j][a] = [(1, i * self.ncol + j, 0,
                                                    True)]
                        continue
                    # 其他位置
                    ## 其他地方的下个状态，或者说下个x坐标，都要满足在格子内部，若是可能移动到网格的外部，下一步就保持不动的
                    next_x = min(self.ncol - 1, max(0, j + change[a][0]))
                    ## 下个状态也就是下个y坐标
                    next_y = min(self.nrow - 1, max(0, i + change[a][1]))
                    ## 下个状态的数组地址
                    next_state = next_y * self.ncol + next_x
                    ## 奖励值初始化到 -1
                    reward = -1
                    ##  默认没有达到终止条件（下个坐标在悬崖，或者在终点）
                    done = False
                    # 下一个位置在悬崖或者终点
                    if next_y == self.nrow - 1 and next_x > 0:    ## 最后一行是悬崖的
                        done = True    ## 达到了终止条件
                        if next_x != self.ncol - 1:  # 下一个位置在悬崖
                            reward = -100    ## 下个坐标是悬崖，奖励是-100
                    ##  下一步的转移概是1，next_state是下一步的坐标，reward是动作奖励，done表示是否终止
                    P[i * self.ncol + j][a] = [(1, next_state, reward, done)]   
        return P

class PolicyIteration:
    """ 策略迭代算法 """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow  # 初始化价值为0
        self.pi = [[0.25, 0.25, 0.25, 0.25]
                   for i in range(self.env.ncol * self.env.nrow)]  # 初始化为均匀随机策略
        self.theta = theta  # 策略评估收敛阈值
        self.gamma = gamma  # 折扣因子

    def policy_evaluation(self):  # 策略评估
        cnt = 1  # 计数器
        while 1:      ## 不断地循环的
            max_diff = 0           ##  最大的不同
            new_v = [0] * self.env.ncol * self.env.nrow             ##  状态价值函数的列表
            for s in range(self.env.ncol * self.env.nrow):          ##  遍历所有的状态
                qsa_list = []  # 开始计算状态s下的所有Q(s,a)价值
                for a in range(4):  ##  遍历所有的动作，上下左右这几个方向
                    qsa = 0         ##  动作价值初始化到0
                    ## 遍历（s,a）对应的s转移可能，拿到状态和动作（s,a）对应的  概率、下个状态的、动作的奖励r、是否终止的
                    ## 上述的code，对应的self.env.P[s][a]的长度都是1，也就是执行某个动作以后，该动作只能转移到某个确定的状态，不能按P转移到多个状态
                    for res in self.env.P[s][a]:
                        p, next_state, r, done = res        ## 状态转移概率默认=1、下个状态的、动作的奖励r(该循环内不变)、是否终止的
                        '''
                        根据公式来算动作价值，使用old价值函数的，当下个状态是终止条件时(1-done)=0，就不算该坐标的回报，也就是后面的部分=0
                        本循环内部 p的累加值=1，所以p1*r+p2*r+...=(p1+p2+...)*r = r，和公式是可以对应起来的
                        也就是贝尔曼期望方程的后半部分
                        '''
                        qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                        # 本章环境比较特殊,奖励和下一个状态有关,所以需要和状态转移概率相乘
                    qsa_list.append(self.pi[s][a] * qsa) ##  还有一个选择动作的可能性 self.pi，每个动作的pi，对应公式贝尔曼期望方程最开始的累加部分
                ##  累加动作价值，得到状态价值，赋值到 new 价值函数的，也就是贝尔曼期望方程最开始的累加部分
                new_v[s] = sum(qsa_list)  # 状态价值函数和动作价值函数之间的关系
                max_diff = max(max_diff, abs(new_v[s] - self.v[s]))  ##  old价值函数和new价值函数的绝对值差值
            self.v = new_v   ##  复制new价值函数给old价值函数
            ## old价值函数和new价值函数的绝对值差值很小了，< 给定的很小值 self.theta
            if max_diff < self.theta: break  # 满足收敛条件,退出评估迭代
            cnt += 1
        print("策略评估进行%d轮后完成" % cnt)

    def policy_improvement(self):  # 策略提升
        for s in range(self.env.nrow * self.env.ncol):          ##  遍历所有的状态
            qsa_list = []  # 开始计算状态s下的所有Q(s,a)价值
            for a in range(4):  ##  遍历所有的动作，上下左右这几个方向
                qsa = 0         ##  动作价值初始化到0
                ## 遍历（s,a）对应的s转移可能，拿到状态和动作（s,a）对应的  概率、下个状态的、动作的奖励r、是否终止的
                ## 上述的code，对应的self.env.P[s][a]的长度都是1，也就是执行某个动作以后，该动作只能转移到某个确定的状态，不能按P转移到多个状态
                for res in self.env.P[s][a]:
                    p, next_state, r, done = res        ## 状态转移概率默认=1、下个状态的、动作的奖励r(该循环内不变)、是否终止的
                    '''
                    根据公式来算动作价值，使用old价值函数的，当下个状态是终止条件时(1-done)=0，就不算该坐标的回报，也就是后面的部分=0
                    本循环内部 p的累加值=1，所以p1*r+p2*r+...=(p1+p2+...)*r = r，和公式是可以对应起来的
                    也就是贝尔曼期望方程的后半部分
                    '''
                    qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                qsa_list.append(qsa)  ## 和策略评估相比较，少了状态动作选择的概率也就是没有策略了，此时选择每个动作的概率都是1
            maxq = max(qsa_list)      ## 拿到这几个动作内最大的价值
            ## 统计最大动作价值的个数，也就是对应的动作个数
            cntq = qsa_list.count(maxq)  # 计算有几个动作得到了最大的Q值
            # 让这些动作均分概率，也就是有最大动作价值的动作，其他动作价值小于最大值的动作，都被删除了也就是动作概率=0
            self.pi[s] = [1 / cntq if q == maxq else 0 for q in qsa_list]
        print("策略提升完成")
        return self.pi
            
    def policy_iteration(self):  # 策略迭代
        while 1:
            self.policy_evaluation()   ## 策略评估函数
            old_pi = copy.deepcopy(self.pi)  # 将列表进行深拷贝,方便接下来进行比较
            new_pi = self.policy_improvement()  ##  策略提升函数
            if old_pi == new_pi: break          ##  策略不变了就可以停止迭代

In [18]:
def print_agent(agent, action_meaning, disaster=[], end=[]):
    print("状态价值：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 为了输出美观,保持输出6个字符
            print('%6.6s' % ('%.3f' % agent.v[i * agent.env.ncol + j]),
                  end=' ')
        print()

    print("策略：")
    for i in range(agent.env.nrow):
        for j in range(agent.env.ncol):
            # 一些特殊的状态,例如悬崖漫步中的悬崖
            if (i * agent.env.ncol + j) in disaster:
                print('****', end=' ')
            elif (i * agent.env.ncol + j) in end:  # 目标状态
                print('EEEE', end=' ')
            else:
                a = agent.pi[i * agent.env.ncol + j]
                pi_str = ''
                for k in range(len(action_meaning)):
                    pi_str += action_meaning[k] if a[k] > 0 else 'o'
                print(pi_str, end=' ')
        print()


env = CliffWalkingEnv()
action_meaning = ['^', 'v', '<', '>']
theta = 0.001
gamma = 0.9
agent = PolicyIteration(env, theta, gamma)
agent.policy_iteration()
print_agent(agent, action_meaning, list(range(37, 47)), [47])

# 策略评估进行60轮后完成
# 策略提升完成
# 策略评估进行72轮后完成
# 策略提升完成
# 策略评估进行44轮后完成
# 策略提升完成
# 策略评估进行12轮后完成
# 策略提升完成
# 策略评估进行1轮后完成
# 策略提升完成
# 状态价值：
# -7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710
# -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900
# -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000
# -7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
# 策略：
# ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo
# ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo
# ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo
# ^ooo **** **** **** **** **** **** **** **** **** **** EEEE

策略评估进行60轮后完成
策略提升完成
策略评估进行72轮后完成
策略提升完成
策略评估进行44轮后完成
策略提升完成
策略评估进行12轮后完成
策略提升完成
策略评估进行1轮后完成
策略提升完成
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


In [25]:
class ValueIteration:
    """ 价值迭代算法 """
    def __init__(self, env, theta, gamma):
        self.env = env
        self.v = [0] * self.env.ncol * self.env.nrow  # 初始化价值为0
        self.theta = theta  # 价值收敛阈值
        self.gamma = gamma
        # 价值迭代结束后得到的策略
        self.pi = [None for i in range(self.env.ncol * self.env.nrow)]

    def value_iteration(self):
        cnt = 0
        while 1:
            max_diff = 0           ##  最大的不同
            new_v = [0] * self.env.ncol * self.env.nrow             ##  状态价值函数的列表
            for s in range(self.env.ncol * self.env.nrow):          ##  遍历所有的状态
                qsa_list = []  # 开始计算状态s下的所有Q(s,a)价值
                for a in range(4):  ##  遍历所有的动作，上下左右这几个方向
                    qsa = 0         ##  动作价值初始化到0
                    ## 遍历（s,a）对应的s转移可能，拿到状态和动作（s,a）对应的  概率、下个状态的、动作的奖励r、是否终止的
                    ## 上述的code，对应的self.env.P[s][a]的长度都是1，也就是执行某个动作以后，该动作只能转移到某个确定的状态，不能按P转移到多个状态
                    for res in self.env.P[s][a]:
                        p, next_state, r, done = res        ## 状态转移概率默认=1、下个状态的、动作的奖励r(该循环内不变)、是否终止的
                        '''
                        根据公式来算动作价值，使用old价值函数的，当下个状态是终止条件时(1-done)=0，就不算该坐标的回报，也就是后面的部分=0
                        本循环内部 p的累加值=1，所以p1*r+p2*r+...=(p1+p2+...)*r = r，和公式是可以对应起来的
                        也就是贝尔曼最优方程的后半部分
                        '''
                        qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                    ## 和策略评估相比较，少了状态动作选择的概率也就是没有策略了，此时选择每个动作的概率都是1
                    qsa_list.append(qsa)  # 这一行和下一行代码是价值迭代和策略迭代的主要区别
                new_v[s] = max(qsa_list)      ## 拿到这几个动作内最大的价值
                max_diff = max(max_diff, abs(new_v[s] - self.v[s]))  ##  old价值函数和new价值函数的绝对值差值
            self.v = new_v   ##  复制new价值函数给old价值函数
            ## old价值函数和new价值函数的绝对值差值很小了，< 给定的很小值 self.theta
            if max_diff < self.theta: break  # 满足收敛条件,退出评估迭代
            cnt += 1
        print("价值迭代一共进行%d轮" % cnt)
        self.get_policy()

    def get_policy(self):  # 根据价值函数导出一个贪婪策略
        for s in range(self.env.nrow * self.env.ncol):
            qsa_list = []  # 开始计算状态s下的所有Q(s,a)价值
            for a in range(4):  ##  遍历所有的动作，上下左右这几个方向
                qsa = 0         ##  动作价值初始化到0
                ## 遍历（s,a）对应的s转移可能，拿到状态和动作（s,a）对应的  概率、下个状态的、动作的奖励r、是否终止的
                ## 上述的code，对应的self.env.P[s][a]的长度都是1，也就是执行某个动作以后，该动作只能转移到某个确定的状态，不能按P转移到多个状态
                for res in self.env.P[s][a]:
                    p, next_state, r, done = res        ## 状态转移概率默认=1、下个状态的、动作的奖励r(该循环内不变)、是否终止的
                    '''
                    根据公式来算动作价值，使用old价值函数的，当下个状态是终止条件时(1-done)=0，就不算该坐标的回报，也就是后面的部分=0
                    本循环内部 p的累加值=1，所以p1*r+p2*r+...=(p1+p2+...)*r = r，和公式是可以对应起来的
                    也就是贝尔曼期望方程的后半部分
                    '''
                    qsa += p * (r + self.gamma * self.v[next_state] * (1 - done))
                qsa_list.append(qsa)  ## 和策略评估相比较，少了状态动作选择的概率也就是没有策略了，此时选择每个动作的概率都是1
            maxq = max(qsa_list)      ## 拿到这几个动作内最大的价值
            ## 统计最大动作价值的个数，也就是对应的动作个数
            cntq = qsa_list.count(maxq)  # 计算有几个动作得到了最大的Q值
            # 让这些动作均分概率
            # 让这些动作均分概率，也就是有最大动作价值的动作，其他动作价值小于最大值的动作，都被删除了也就是动作概率=0
            self.pi[s] = [1 / cntq if q == maxq else 0 for q in qsa_list]

env = CliffWalkingEnv()
action_meaning = ['^', 'v', '<', '>']
theta = 0.001
gamma = 0.9
agent = ValueIteration(env, theta, gamma)
agent.value_iteration()
print_agent(agent, action_meaning, list(range(37, 47)), [47])

# 价值迭代一共进行14轮
# 状态价值：
# -7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710
# -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900
# -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000
# -7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
# 策略：
# ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo
# ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo
# ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo
# ^ooo **** **** **** **** **** **** **** **** **** **** EEEE

价值迭代一共进行14轮
状态价值：
-7.712 -7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 
-7.458 -7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 
-7.176 -6.862 -6.513 -6.126 -5.695 -5.217 -4.686 -4.095 -3.439 -2.710 -1.900 -1.000 
-7.458  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 
策略：
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovo> ovoo 
ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ooo> ovoo 
^ooo **** **** **** **** **** **** **** **** **** **** EEEE 


In [26]:
import gymnasium as gym
env = gym.make("FrozenLake-v1")  # 创建环境
_ = env.reset(seed=0)
env = env.unwrapped  # 解封装才能访问状态转移矩阵P
env.render()  # 环境渲染,通常是弹窗显示或打印出可视化的环境

holes = set()
ends = set()
for s in env.P:  ##  遍历环境的所有状态
    for a in env.P[s]:  ##  遍历状态下所有的动作
        for s_ in env.P[s][a]:  ##  遍历每个动作对应的转移情况
            if s_[2] == 1.0:  # 获得奖励为1,代表是目标
                ends.add(s_[1]) ##  终止标号的呢
            if s_[3] == True:   ##  是否洞
                holes.add(s_[1])  ## 标号
holes = holes - ends
print("冰洞的索引:", holes)
print("目标的索引:", ends)

## 共有三个动作的，每个动作有三种转移可能
## 每个动作都有三种状态转移的可能，和上面的悬崖有些区别的，上面的悬崖只有一种可能
## 导致这种情况的reason，就是冰面太滑了，可能滑到其他地方去了
for a in env.P[14]:  # 查看目标左边一格的状态转移信息
    print(env.P[14][a])

# SFFF
# FHFH
# FFFH
# HFFG
# 冰洞的索引: {11, 12, 5, 7}
# 目标的索引: {15}
# [(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False),
#  (0.3333333333333333, 14, 0.0, False)]
# [(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False),
#  (0.3333333333333333, 15, 1.0, True)]
# [(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True),
#  (0.3333333333333333, 10, 0.0, False)]
# [(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False),
#  (0.3333333333333333, 13, 0.0, False)]

冰洞的索引: {11, 12, 5, 7}
目标的索引: {15}
[(0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False)]
[(0.3333333333333333, 13, 0.0, False), (0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True)]
[(0.3333333333333333, 14, 0.0, False), (0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False)]
[(0.3333333333333333, 15, 1.0, True), (0.3333333333333333, 10, 0.0, False), (0.3333333333333333, 13, 0.0, False)]


In [29]:
# 这个动作意义是Gym库针对冰湖环境事先规定好的
action_meaning = ['<', 'v', '>', '^']   ## 上下左右的标识符
theta = 1e-5   ## 当绝对值差值小于阀值时，停止迭代的呢
gamma = 0.9    ## γ值
agent = PolicyIteration(env, theta, gamma)  ## 实例化策略迭代
agent.policy_iteration()      ## 迭代
print_agent(agent, action_meaning, [5, 7, 11, 12], [15]) ## 输出

# 策略评估进行25轮后完成
# 策略提升完成
# 策略评估进行58轮后完成
# 策略提升完成
# 状态价值：
#  0.069  0.061  0.074  0.056
#  0.092  0.000  0.112  0.000
#  0.145  0.247  0.300  0.000
#  0.000  0.380  0.639  0.000
# 策略：
# <ooo ooo^ <ooo ooo^
# <ooo **** <o>o ****
# ooo^ ovoo <ooo ****
# **** oo>o ovoo EEEE

策略评估进行25轮后完成
策略提升完成
策略评估进行58轮后完成
策略提升完成
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 


In [8]:
action_meaning = ['<', 'v', '>', '^']   ## 上下左右的标识符
theta = 1e-5   ## 当绝对值差值小于阀值时，停止迭代的呢
gamma = 0.9    ## γ值
agent = ValueIteration(env, theta, gamma)  ## 实例化价值迭代
agent.value_iteration()      ## 迭代
print_agent(agent, action_meaning, [5, 7, 11, 12], [15]) ## 输出

# 价值迭代一共进行60轮
# 状态价值：
#  0.069  0.061  0.074  0.056
#  0.092  0.000  0.112  0.000
#  0.145  0.247  0.300  0.000
#  0.000  0.380  0.639  0.000
# 策略：
# <ooo ooo^ <ooo ooo^
# <ooo **** <o>o ****
# ooo^ ovoo <ooo ****
# **** oo>o ovoo EEEE

价值迭代一共进行60轮
状态价值：
 0.069  0.061  0.074  0.056 
 0.092  0.000  0.112  0.000 
 0.145  0.247  0.300  0.000 
 0.000  0.380  0.639  0.000 
策略：
<ooo ooo^ <ooo ooo^ 
<ooo **** <o>o **** 
ooo^ ovoo <ooo **** 
**** oo>o ovoo EEEE 
